In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


In [26]:
removeNtrials = 0
divideIntoNparts = 8

In [27]:
absoluteSize = False # True means absolute, False means relative

In [28]:
KeyInclude = ['IncreaseTimeDwell', 'DecreaseTimeDwell']
TimeDwellOrig = 800
TimeFixation = 300

In [29]:
def filterBlinks(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    
   # Blink detection    
    missingPupilData = dict()
    missingPupilData['Start'] = list()
    missingPupilData['End'] = list()
    missingPupilData['Duration'] = list()
    missingPupilData['DurationExtra'] = list()
    
    pupilData_filter = list()
    time_filter = list()
    index_blinkEnd = list()
    
    missingBeginning = 0
    missingEnd = 0

    # First, find the missing values
    pupilInd = -1
    for pupilD in pupilData:
        
        pupilInd = pupilInd + 1
        #if pupilInd>772 and pupilInd<940:
        #    print(pupilInd, pupilD)
            
        # check if val is nan
        if np.isnan(pupilD):
            # check if first element of list
            if pupilInd == 0:
                # first value is nan
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
            # check if previous val was nan
            elif np.isnan(pupilData[pupilInd-1]):
                
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingEnd = 1
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
                    
            # if new nan, note the index
            else:
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
                # for single nan values
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
            
    
    
    # Second, create a list that does not contain missing values or 200ms before and after that
    
     # blink is every nan value in the range of 100-400ms 
    # 200 ms (18 samples) before and after the blink will also be removed
    addedBlinkSamples = 18    
    blinkDurationStart = 0
    blinkDurationEnd = 0
    
    iCurrent = len(missingPupilData['Start'])
    endCurrent = iCurrent - 1
    isBlink = 0
    pupilData_filter = list(pupilData)
    time_filter = list(timeListComplete)
    
    index_blinkEndExtra = list() # list to document if 2 blinks joined 
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        
        #print('Current', missingPupilData['Start'][iCurrent], missingPupilData['End'][iCurrent])
        
        # check if it is really a blink, and remove if it is not
        # if blink duration is less than 100ms, the reason for missing data is something else
        timeStart0 = time_filter[missingPupilData['Start'][iCurrent]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        timeEnd0 = time_filter[missingPupilData['End'][iCurrent]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        time1 = (timeEnd - timeStart).total_seconds()
        #print(time1)
        if time1 < 0.1 or time1 > 1.5:
            isNotBlink = 1
        else:
            isNotBlink = 0
            
        # Remove if it is a blink
        if isNotBlink:
            del pupilData_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            del time_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            
            # length of non-blink
            lengthBlink = missingPupilData['End'][iCurrent] - missingPupilData['Start'][iCurrent] + 1
            
            # all blinks after this non-blink need to be reduced in indexes
            #print(len(missingPupilData['Start'])-1)
            
            for iChange in range(iCurrent-1, len(missingPupilData['Start'])-1):
                #print('i', iChange)
                if iChange == iCurrent-1:
                    
                    #print('Deleted', missingPupilData['Start'][iChange+1], missingPupilData['End'][iChange+1])
                    #print(lengthBlink)
                    del missingPupilData['Start'][iChange+1]
                    del missingPupilData['End'][iChange+1]
                else:
                    
                    #print('Old', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    missingPupilData['Start'][iChange] = missingPupilData['Start'][iChange] - lengthBlink
                    missingPupilData['End'][iChange] = missingPupilData['End'][iChange] - lengthBlink
                    #print('New', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    
    #print(len(missingPupilData['Start']))
    
    previousBlinkEnd = 0
    nextBlinkEnd = 0
    iCurrent = len(missingPupilData['Start'])
    
#     for index, (value1, value2) in enumerate(zip(missingPupilData['Start'], missingPupilData['End'])):
#         print(index, value1, value2)
#     for i in enumerate(pupilData_filter):
#         print(i)
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        #print(iCurrent)
        iNow = iCurrent # iCurrent is modified in advance later (possibly), which will disturb the current data index
        #print(missingPupilData['Start'][iNow], missingPupilData['End'][iNow])
        
#         if iNow > 1 :
#             print('BEFORE!!!!!!!!!!!!!!!!!')
#             print(pupilData_filter[missingPupilData['Start'][iNow]:-1])
            
        # to make sure that blink removal data does not go into the previous data
        if iCurrent == 0:
            previousBlinkEnd = 0
        else:
            previousBlinkEnd = missingPupilData['Start'][iCurrent-1]

        # to make sure blink does not go into the next one
        if iCurrent == len(missingPupilData['Start'])-1:
            #print(iCurrent)
            nextBlinkEnd = len(pupilData_filter)
            #print(nextBlinkEnd)
        else:
            nextBlinkEnd = missingPupilData['Start'][iCurrent+1]
            
        
        
        if missingPupilData['End'][iCurrent] + addedBlinkSamples > nextBlinkEnd:
            #LOOK INTO THIS, NEXT BLINK IS REMOVED BUT ITS EXTRA 200MS IS NOT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            #print('After', missingPupilData['End'][iCurrent], addedBlinkSamples, '>>>>>', nextBlinkEnd)
            blinkRemoveAfter = abs(missingPupilData['End'][iCurrent] - nextBlinkEnd)
        else:
            blinkRemoveAfter = addedBlinkSamples
        
        if missingPupilData['Start'][iCurrent] - addedBlinkSamples < previousBlinkEnd:
            #print('Before', missingPupilData['Start'][iCurrent], addedBlinkSamples, '<<<<<<', previousBlinkEnd)
            blinkRemoveBefore = abs(missingPupilData['Start'][iCurrent] - previousBlinkEnd)
            
            # Add the extra duration to list
            if iCurrent > 0:
                
                timeStart0 = time_filter[missingPupilData['Start'][iCurrent-1]].partition('+')
                timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                timeEnd0 = time_filter[missingPupilData['End'][iCurrent-1]].partition('+')
                timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                time1 = (timeEnd - timeStart).total_seconds()
                #print(timeEnd, timeStart, time1)

                missingPupilData['DurationExtra'].append(time1)
#             if iCurrent > 1:
#                 print(missingPupilData['Start'][iCurrent-1], 'also removed')
#             # After next blink is also removed, 
            iCurrent = iCurrent - 1
            index_blinkEndExtra.append(2)
            
                
        else:
            blinkRemoveBefore = addedBlinkSamples
            index_blinkEndExtra.append(1)
            missingPupilData['DurationExtra'].append(0)
            
#         print(missingPupilData['Start'][iNow]-blinkRemoveBefore)
        
        timeStart0 = time_filter[missingPupilData['Start'][iNow]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeStart, pupilData_filter[missingPupilData['Start'][iNow]])     
        timeEnd0 = time_filter[missingPupilData['End'][iNow]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeEnd) 
        time2 = (timeEnd - timeStart).total_seconds()
        #print(time2)
        missingPupilData['Duration'].append(time2)
        del pupilData_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        del time_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        
        index_blinkEnd.append(missingPupilData['Start'][iNow] - blinkRemoveBefore - 1)
    #         if iNow > 1 :
#             print('AFTER!!!!!!!!!!!!')
#             if np.nan in pupilData_filter[missingPupilData['Start'][iNow]:-1]:
#                 for i,j in enumerate(pupilData_filter[missingPupilData['Start'][iNow]:-1]):
#                     print(i+missingPupilData['End'][iNow]+blinkRemoveAfter,j)
        
    
    if np.nan in pupilData_filter:
        for i in enumerate(pupilData_filter):
            print(i)
     
    index_blinkEnd = list(reversed(index_blinkEnd))
    
    return pupilData_filter, time_filter, index_blinkEnd, index_blinkEndExtra, missingPupilData

In [30]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [31]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtNewTrial(UserLooksAtKey):
    # Create list of list (made of epochs) composed of times when user looked at NextPhrase
    
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstNextPhrase = 0
    flagStart = 0
    firstTrial = 0
    
    # Find unique looks at new trial, by checking if progress pct is greater than previous one and store first look in a list
    
    progressNextPhrase = 0 
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        # first add data for first trial, before first next phrase is reached
        if firstTrial == 0:
            firstTrial = 1
            epochList.append([row, 0])
            ##print(epochList)
            continue
            
        if row[1] == 'NextPhrase':
            
            # first eliminate an ongoing look
            if UserLooksAtKey[nRow-1][1] == 'NextPhrase':
                continue
            else:
                # normally for new looks
                
                # if it is a continued look, progressNextPhrase should not be 0
                ##print('Continued ???', progressNextPhrase, UserLooksAtKey[nRow])
                
                if progressNextPhrase > float(UserLooksAtKey[nRow][2]):
                    
                    # save it as 0, if not a continued look
                    ##print('progress is now 0')
                    progressNextPhrase = 0
#                 else:
#                     if progressNextPhrase == float(UserLooksAtKey[nRow][2]):
#                         #print('ITS EQUAL!!!')
                    
                
            # now check if this look is going to be successfully selected:
            nextPhrase_selected = 0
            rowCurrent = nRow-1
            
            while nextPhrase_selected < 1:
                rowCurrent = rowCurrent + 1
                if UserLooksAtKey[rowCurrent][1] == 'NextPhrase':
                    if float(UserLooksAtKey[rowCurrent][2]) == 1:
                        ##print(UserLooksAtKey[rowCurrent])
                        nextPhrase_selected = 1
                        rowCurrentEnd = rowCurrent
                        
                        
                        if progressNextPhrase != 0:
                            epochList[-1].insert(2, UserLooksAtKey[rowPreviousEnd])
                            epochList.append([UserLooksAtKey[rowCurrentStart], UserLooksAtKey[rowCurrentEnd]])
                        else:
                            epochList[-1].insert(2, UserLooksAtKey[nRow-1])
                            epochList.append([UserLooksAtKey[nRow], UserLooksAtKey[rowCurrentEnd]])
                    else:
                        continue
                else:
                    # if look at next phrase is stopped, store the previous look progress, in case it is continued later
                    progressNextPhrase = float(UserLooksAtKey[rowCurrent-1][2])
                    rowCurrentStart = nRow
                    rowPreviousEnd = nRow - 1
                    break
                    
                     
                    
                    
    # insert last element before quiting - 
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in UserLooksAtKey]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
    
    epochList[-1].insert(2, UserLooksAtKey[indBeforeQuit])
    
    iRemove = removeNtrials
    while iRemove > 0:
        iRemove = iRemove - 1
        epochList.remove(epochList[iRemove])
    
#     for i in epochList:
#         print(i)
#     print('next')
    return epochList[0][0][0]

In [32]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [33]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [34]:
def SplitDataIntoParts(timeStartUserKeys, timeEndScratchPad, timeFiltered):
    
    # find time of expt (2nd half of it)
    timeStart, timeStartInd = nearestTimePoint(timeFiltered, timeStartUserKeys)
    timeEnd, timeEndInd = nearestTimePoint(timeFiltered, timeEndScratchPad)
    timeInS = (timeEnd - timeStart).total_seconds()
    
    # divide into 2/3
    timeEachPart = datetime.timedelta(seconds=timeInS/divideIntoNparts)
    
    timePartsList = [[] for i in range(2)]
    indexPartsList = [[] for i in range(2)]
    
    indexPartsList[0].append(timeStartInd)
    
    # find start and end times of each part
    for i in range(divideIntoNparts):

        timeNew = timeStart + timeEachPart

        if i == divideIntoNparts - 1:
            timePartsList[0].append(timeStart)
            timeEnd1 = timeEnd
            timePartsList[1].append(timeEnd)
            indexPartsList[1].append(timeEndInd)
            
            break
        else:
            timePartsList[0].append(timeStart)
            timeEnd1, timeEnd1Ind = nearestTimePoint(timeFiltered, timeNew)
            timeStart = timeFiltered[timeEnd1Ind+1]
            
            timePartsList[1].append(timeEnd1)
            indexPartsList[1].append(timeEnd1Ind)
            # for the next part
            indexPartsList[0].append(timeEnd1Ind+1)
    
#    print(len(timePartsList[0]))
    return timePartsList, indexPartsList
        
    


In [35]:
# find end time of Expt - when the scratchpad is erased, at the end
def FindTimeEnd(scratchPadList):
    
    scratchPadText = [i[1] for i in scratchPadList]
    scratchPadTime = [i[0] for i in scratchPadList]
                
    iText = len(scratchPadText)
    while iText > 0:
        iText = iText - 1
        if scratchPadText[iText] == '':
            if scratchPadText[iText-1] != '':
                findTimeAtIndex = iText
                break
                    
    return scratchPadTime[findTimeAtIndex]

In [36]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx] # smoothing
    
    return(vals)

In [77]:
def ComputeAveragePupilSizeForEveryPart(pupilLogL_filter, pupilLogR_filter, timeListParts, indexListParts, subjName):
    
    pupilL_avgList = list()
    pupilR_avgList = list()
    
    winSize = 40
    # hampel filter on whole data
    pupilLogL_filterWoOutlier = hampel(pupilLogL_filter, winSize, 3)
    pupilLogR_filterWoOutlier = hampel(pupilLogR_filter, winSize, 3)
    
    pupilAbsoluteL = pupilLogL_filterWoOutlier.rolling(window=winSize, min_periods=1, center=True).mean()
    pupilAbsoluteR = pupilLogR_filterWoOutlier.rolling(window=winSize, min_periods=1, center=True).mean()
            
    # find reference pupil size
    pupilL_ref = np.mean(pupilAbsoluteL[0:50])    
    pupilR_ref = np.mean(pupilAbsoluteR[0:50])
    
    # for every part, find average relative pupil size
    for iPart in range(len(timeListParts[0])):
#        print(iPart)
        pupilL = pupilAbsoluteL[indexListParts[0][iPart]:indexListParts[1][iPart]]
        pupilR = pupilAbsoluteR[indexListParts[0][iPart]:indexListParts[1][iPart]]
        
        pupilL_relative = [(i-pupilL_ref[0])/pupilL_ref[0] for i in pupilL[0]]
        pupilR_relative = [(i-pupilR_ref[0])/pupilR_ref[0] for i in pupilR[0]]
        
#        print(len(pupilL_relative))
        
        pupilL_avg = np.mean(pupilL_relative)
        pupilR_avg = np.mean(pupilR_relative)
        
        pupilL_avgList.append(pupilL_avg)
        pupilR_avgList.append(pupilR_avg)
        
    # plot the average relative pupil size
    
    #print(len(pupilL_avgList))

    
    return pupilL_avgList, pupilR_avgList
    
    

In [78]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
i = 0
flagFirstSubj = 0
pupil_avgParts = dict()

# pupil_avgParts['novice_right'] = [[] for i in range(divideIntoNparts)]
# pupil_avgParts['expert_right'] = [[] for i in range(divideIntoNparts)]
# pupil_avgParts['novice_left'] = [[] for i in range(divideIntoNparts)]
# pupil_avgParts['expert_left'] = [[] for i in range(divideIntoNparts)]

pupil_avgParts['DT_right'] = [[] for i in range(divideIntoNparts)]
pupil_avgParts['MS_right'] = [[] for i in range(divideIntoNparts)]
pupil_avgParts['DT_left'] = [[] for i in range(divideIntoNparts)]
pupil_avgParts['MS_left'] = [[] for i in range(divideIntoNparts)]

# novices = ['be_DT', 'kea_MS', 'lone_DT', 'mcc_MS', 'mk_DT', 'ok_MS', 'pt_DT', 'slh_DT', 'ae_DT', 'js_MS', 'ma_DT', 'mw_MS', 'pgba_DT', 'smn_DT', 'jg_DT', 'mr_DT', 'snk_DT']

for root, dirs, files in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root or 'notEnough' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root or 'ae' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in files:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
                        
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
                        
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or gazeLog is None:
                
                continue
            else:
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                typing_mechanism = subjName[-2:]
                print(subjName)
                
                # obtain right and left pupil data
                # Create list of pupil sizes from gazelog
                pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                timeList = [item[0] for item in gazeLog]
                
                # filter the blinks
                pupilLogL_filter, time_filter, index_blinkEndL, index_blinkEndExtra, missingDataL = filterBlinks(pupilLogL, timeList)
                
                pupilLogR_filter, time_filter, index_blinkEndR, index_blinkEndExtra, missingDataR = filterBlinks(pupilLogR, timeList)
                
                time_filterFormatted = timeConversion(time_filter)
                
                # Compute dwell time
                userKeysWTime = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                timeStartTrial3Str = UserLookedAtNewTrial(userKeysWTime)
                timeStartTrial3Str1, t1, t2 = timeStartTrial3Str.partition('+')
                timeStartTrial3UserKeys = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStartTrial3Str1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                
                timeExptEndStr = FindTimeEnd(scratchPad)
                timeExptEndStr1, t1, t2 = timeExptEndStr.partition('+')
                timeExptEndScratchPad = (datetime.datetime.strptime(re.sub('[:.T]','-',timeExptEndStr1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                # Determine when the second part starts
                timeListParts, indexListParts = SplitDataIntoParts(timeStartTrial3UserKeys, timeExptEndScratchPad, time_filterFormatted)
                
                # For every part, find relative pupil size average
                pupilL_avgList, pupilR_avgList = ComputeAveragePupilSizeForEveryPart(pupilLogL_filter, pupilLogR_filter, timeListParts, indexListParts, subjName)
                
                if 'MS' in subjName:
                    
                    for i in range(len(pupilL_avgList)):
                        
                        pupil_avgParts['MS_left'][i].append(pupilL_avgList[i])
                        pupil_avgParts['MS_right'][i].append(pupilR_avgList[i])
                        
                else:
                    for i in range(len(pupilL_avgList)):
                        
                        pupil_avgParts['DT_left'][i].append(pupilL_avgList[i])
                        pupil_avgParts['DT_right'][i].append(pupilR_avgList[i])
                    
#                 # compare novices to others
                
#                 if subjName in novices:
#                     for i in range(len(pupilL_avgList)):
                        
#                         pupil_avgParts['novice_left'][i].append(pupilL_avgList[i])
#                         pupil_avgParts['novice_right'][i].append(pupilR_avgList[i])
#                 else:
#                     for i in range(len(pupilL_avgList)):
                        
#                         pupil_avgParts['expert_left'][i].append(pupilL_avgList[i])
#                         pupil_avgParts['expert_right'][i].append(pupilR_avgList[i])        
        

akt_MS
KEA_MS
mcc_MS
MK_DT
ok_MS
pt_DT
sh_MS
hc_MS
ib_MS
km_DT
ma_DT
pgba_DT
smn_DT
jek_MS
lr_MS
mm_MS
snk_DT


In [69]:
removeNtrials = 0
divideIntoNparts = 10

In [82]:
fig = plt.figure()
plt.plot(pupil_avgParts['DT'])

fig = plt.figure()
plt.plot(pupil_avgParts['MS'])

In [80]:
pupil_avgParts['DT'] = [[] for i in range(divideIntoNparts)]
pupil_avgParts['MS'] = [[] for i in range(divideIntoNparts)]

for iList in range(len(pupil_avgParts['DT_left'])):
    for i in range(len(pupil_avgParts['DT_left'][iList])):
        pupil_avgParts['DT'][iList].append(pupil_avgParts['DT_left'][iList][i] + pupil_avgParts['DT_right'][iList][i])

for iList in range(len(pupil_avgParts['MS_left'])):
    for i in range(len(pupil_avgParts['MS_left'][iList])):
        pupil_avgParts['MS'][iList].append(pupil_avgParts['MS_left'][iList][i] + pupil_avgParts['MS_right'][iList][i])

        

In [68]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

xAxis = range(1,divideIntoNparts+1)

#ax.plot(xAxis, np.mean(pupil_avgParts['DT_right'],1))
ax.errorbar(xAxis, np.mean(pupil_avgParts['DT'],1), np.std(pupil_avgParts['DT'],1), linestyle='None', marker='^')

#ax.plot(xAxis, np.mean(pupil_avgParts['MS_right'],1))
ax.errorbar(xAxis, np.mean(pupil_avgParts['MS'],1), np.std(pupil_avgParts['MS'],1), linestyle='None', marker='^')


<Container object of 3 artists>

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

xAxis = range(divideIntoNparts)

ax.plot(xAxis, np.mean(pupil_avgParts['novice_left'],1))
ax.fill_between(xAxis, np.mean(pupil_avgParts['novice_left'],1)+np.std(pupil_avgParts['novice_left'],1), np.mean(pupil_avgParts['novice_left'],1)-np.std(pupil_avgParts['novice_left'],1), alpha = 0.3)

ax.plot(xAxis, np.mean(pupil_avgParts['expert_left'],1))
ax.fill_between(xAxis, np.mean(pupil_avgParts['expert_left'],1)+np.std(pupil_avgParts['expert_left'],1), np.mean(pupil_avgParts['expert_left'],1)-np.std(pupil_avgParts['expert_left'],1), alpha = 0.3)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

xAxis = range(divideIntoNparts)

ax.plot(xAxis, np.mean(pupil_avgParts['novice_right'],1))
ax.fill_between(xAxis, np.mean(pupil_avgParts['novice_right'],1)+np.std(pupil_avgParts['novice_right'],1), np.mean(pupil_avgParts['novice_right'],1)-np.std(pupil_avgParts['novice_right'],1), alpha = 0.3)

ax.plot(xAxis, np.mean(pupil_avgParts['expert_right'],1))
ax.fill_between(xAxis, np.mean(pupil_avgParts['expert_right'],1)+np.std(pupil_avgParts['expert_right'],1), np.mean(pupil_avgParts['expert_right'],1)-np.std(pupil_avgParts['expert_right'],1), alpha = 0.3)

In [ ]:
plt.plot(np.mean(pupil_avgParts['DT_right'],1))
plt.plot(np.std(pupil_avgParts['DT_right'],1))

In [ ]:
plt.plot(np.mean(pupil_avgParts['MS_left'],1))
plt.plot(np.std(pupil_avgParts['MS_left'],1))

In [ ]:
plt.plot(np.mean(pupil_avgParts['MS_right'],1))
plt.plot(np.std(pupil_avgParts['MS_right'],1))

In [ ]:
pupilDataPartOne['CorrectLeft']/pupilDataPartOne['CorrectSubject']
pupilDataPartOne['CorrectRight']/pupilDataPartOne['CorrectSubject']
pupilDataPartOne['CorrectLeftSquared'] = ((pupilDataPartOne['CorrectLeftSquared'])/pupilDataPartOne['CorrectSubject']-(pupilDataPartOne['CorrectLeft'])**2)**0.5
pupilDataPartOne['CorrectRightSquared'] = ((pupilDataPartOne['CorrectRightSquared'])/pupilDataPartOne['CorrectSubject']-(pupilDataPartOne['CorrectRight'])**2)**0.5
pupilDataPartOne['IncorrectLeftSquared'] = ((pupilDataPartOne['IncorrectLeftSquared'])/pupilDataPartOne['IncorrectSubject']-(pupilDataPartOne['IncorrectLeft'])**2)**0.5
pupilDataPartOne['IncorrectRightSquared'] = ((pupilDataPartOne['IncorrectRightSquared'])/pupilDataPartOne['IncorrectSubject']-(pupilDataPartOne['IncorrectRight'])**2)**0.5

pupilDataPartTwo['CorrectLeft']/pupilDataPartTwo['CorrectSubject']
pupilDataPartTwo['CorrectRight']/pupilDataPartTwo['CorrectSubject']
pupilDataPartTwo['CorrectLeftSquared'] = ((pupilDataPartTwo['CorrectLeftSquared'])/pupilDataPartTwo['CorrectSubject']-(pupilDataPartTwo['CorrectLeft'])**2)**0.5
pupilDataPartTwo['CorrectRightSquared'] = ((pupilDataPartTwo['CorrectRightSquared'])/pupilDataPartTwo['CorrectSubject']-(pupilDataPartTwo['CorrectRight'])**2)**0.5
pupilDataPartTwo['IncorrectLeftSquared'] = ((pupilDataPartTwo['IncorrectLeftSquared'])/pupilDataPartTwo['IncorrectSubject']-(pupilDataPartTwo['IncorrectLeft'])**2)**0.5
pupilDataPartTwo['IncorrectRightSquared'] = ((pupilDataPartTwo['IncorrectRightSquared'])/pupilDataPartTwo['IncorrectSubject']-(pupilDataPartTwo['IncorrectRight'])**2)**0.5


In [ ]:
pupilDataPartTwo

In [ ]:
if pupilData:
    fig = plt.figure()
    axL = fig.add_subplot(2, 1, 1)
    axR = fig.add_subplot(2, 1, 2)
    
    axL.hist()

In [ ]:
if pupilData:
    # if the correct and incorrect data are of different sizes,
    pupilSizeMin = min(len(pupilData['CorrectLeft']), len(pupilData['IncorrectLeft']))
    fig = plt.figure()
    axL = fig.add_subplot(2, 1, 1)
    axR = fig.add_subplot(2, 1, 2)
    xAxis = np.arange(0, float(pupilSizeMin/90), float(1/90))
    pupilPlot = dict()
    pupilPlot['CorrectLeft'] = [x/pupilData['CorrectNumber'] for x in pupilData['CorrectLeft']]
    pupilPlot['CorrectRight'] = [x/pupilData['CorrectNumber'] for x in pupilData['CorrectRight']]
    pupilPlot['IncorrectLeft'] = [x/pupilData['IncorrectNumber'] for x in pupilData['IncorrectLeft']]
    pupilPlot['IncorrectRight'] = [x/pupilData['IncorrectNumber'] for x in pupilData['IncorrectRight']]
    
    # Standard deviation
    pupilPlot['CorrectLeftSquared'] = [pupilData['CorrectLeftSquared'][xInd]/pupilData['CorrectNumber']-pupilPlot['CorrectLeft'][xInd] for xInd in range(0, len(pupilData['CorrectLeftSquared']))]
    pupilPlot['CorrectRightSquared'] = [pupilData['CorrectRightSquared'][xInd]/pupilData['CorrectNumber']-pupilPlot['CorrectRight'][xInd] for xInd in range(0, len(pupilData['CorrectRightSquared']))]
    pupilPlot['IncorrectLeftSquared'] = [pupilData['IncorrectLeftSquared'][xInd]/pupilData['IncorrectNumber']-pupilPlot['IncorrectLeft'][xInd] for xInd in range(0, len(pupilData['IncorrectLeftSquared']))]
    pupilPlot['IncorrectRightSquared'] = [pupilData['IncorrectRightSquared'][xInd]/pupilData['IncorrectNumber']-pupilPlot['IncorrectRight'][xInd] for xInd in range(0, len(pupilData['IncorrectRightSquared']))]

    pupilPlot['CorrectLeftStdPositive'] = [pupilPlot['CorrectLeft'][xInd] + pupilPlot['CorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectLeft']))]
    pupilPlot['CorrectLeftStdNegative'] = [pupilPlot['CorrectLeft'][xInd] - pupilPlot['CorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectLeft']))]
    pupilPlot['IncorrectLeftStdPositive'] = [pupilPlot['IncorrectLeft'][xInd] + pupilPlot['IncorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectLeft']))]
    pupilPlot['IncorrectLeftStdNegative'] = [pupilPlot['IncorrectLeft'][xInd] - pupilPlot['IncorrectLeftSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectLeft']))]

    pupilPlot['CorrectRightStdPositive'] = [pupilPlot['CorrectRight'][xInd] + pupilPlot['CorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectRight']))]
    pupilPlot['CorrectRightStdNegative'] = [pupilPlot['CorrectRight'][xInd] - pupilPlot['CorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['CorrectRight']))]
    pupilPlot['IncorrectRightStdPositive'] = [pupilPlot['IncorrectRight'][xInd] + pupilPlot['IncorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectRight']))]
    pupilPlot['IncorrectRightStdNegative'] = [pupilPlot['IncorrectRight'][xInd] - pupilPlot['IncorrectRightSquared'][xInd] for xInd in range(0, len(pupilPlot['IncorrectRight']))]


    axL.plot(xAxis, pupilPlot['CorrectLeft'][0:pupilSizeMin], 'bo', label = 'Correct')
    axL.plot(xAxis, pupilPlot['IncorrectLeft'][0:pupilSizeMin], 'ro', label = 'Incorrect')
    axL.plot(xAxis, pupilPlot['CorrectLeftStdPositive'][0:pupilSizeMin], ':', color = 'blue')
    axL.plot(xAxis, pupilPlot['CorrectLeftStdNegative'][0:pupilSizeMin], ':', color = 'blue')
    axL.plot(xAxis, pupilPlot['IncorrectLeftStdPositive'][0:pupilSizeMin], ':', color = 'red')
    axL.plot(xAxis, pupilPlot['IncorrectLeftStdNegative'][0:pupilSizeMin], ':', color = 'red')
    
    axL.set_title('Left')
    axL.set_xlabel('Time [in s]')
    if absoluteSize:
        axL.set_ylabel('Absolute pupil size [in mm]')
    else:
        axL.set_ylabel('Relative pupil size [in %]')
    axL.legend()

    axR.plot(xAxis, pupilPlot['CorrectRight'][0:pupilSizeMin], 'bo', label = 'Correct')
    axR.plot(xAxis, pupilPlot['IncorrectRight'][0:pupilSizeMin], 'ro', label = 'Incorrect')
    axR.plot(xAxis, pupilPlot['CorrectRightStdPositive'][0:pupilSizeMin], ':', color = 'blue')
    axR.plot(xAxis, pupilPlot['CorrectRightStdNegative'][0:pupilSizeMin], ':', color = 'blue')
    axR.plot(xAxis, pupilPlot['IncorrectRightStdPositive'][0:pupilSizeMin], ':', color = 'red')
    axR.plot(xAxis, pupilPlot['IncorrectRightStdNegative'][0:pupilSizeMin], ':', color = 'red')

    axR.legend()
    axR.set_xlabel('Time [in s]')
    if absoluteSize:
        axR.set_ylabel('Absolute pupil size [in mm]')
    else:
        axR.set_ylabel('Relative pupil size [in %]')


In [ ]:
def maxPhrasesPossible(phrases):
    # function to find max stimulation phrases possible for the user
    
    phrasesOnly = [row[1] for row in phrases]
    if 'THE EXPERIMENT IS NOW DONE' in phrasesOnly:
        phrasesOnly.remove(phrasesOnly[-1])
        
    return len(phrasesOnly)